In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../Dataset/store-sales-time-series-forecasting/train.csv')
test = pd.read_csv('../Dataset/store-sales-time-series-forecasting/test.csv')
store = pd.read_csv('../Dataset/store-sales-time-series-forecasting/stores.csv')
oil = pd.read_csv('../Dataset/store-sales-time-series-forecasting/oil.csv')
holidays_events = pd.read_csv('../Dataset/store-sales-time-series-forecasting/holidays_events.csv')
transaction = pd.read_csv('../Dataset/store-sales-time-series-forecasting/transactions.csv')

In [ ]:
train.info()

In [ ]:
train

In [ ]:
test

In [ ]:
# 商品ジャンルの一覧を出す
train["family"].value_counts()

In [ ]:
# 商品ジャンルの種類の数を出す
train["family"].nunique()

In [ ]:
# 各商品ジャンルの数が90936と綺麗に一致するのが気になる
print(33*90936)

In [ ]:
# 各商品がどれくらい売り上げるのか
pd.options.display.float_format = '{:.2f}'.format
print("Sales amount: " + str(train["sales"].sum()))
train[["family", "sales"]].groupby("family").sum().sort_values("sales", ascending=False)

In [ ]:
oil["date"] = pd.to_datetime(oil["date"])

date_range = pd.date_range(start='1/1/2013', end='8/31/2017', freq='D')
date_range = pd.DataFrame(date_range, columns=["date"])
oil = date_range.merge(oil, on="date", how="left")

In [ ]:
oil.set_index('date').interpolate(limit=3, limit_direction='backward').plot()

In [ ]:
# 2013年～2017年の各月ごとの売り上げ推移をみる
train_info = train[["date", "family", "sales"]]
train_info["date"] = pd.to_datetime(train_info["date"])
train_info["date_monthly"] = train_info['date'].dt.strftime('%Y-%m')
# train_info = train_info.set_index("date")
# train_info.resample("M").sum()
sales_family_monthly = pd.pivot_table(train_info, values='sales', index='family', columns='date_monthly', aggfunc='sum', fill_value=0)
sales_family_monthly

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (20,10))
plt.xticks(rotation=90)
plt.ticklabel_format(style='plain',axis='y')
# sns.relplot(x='date_monthly', y='family', data=sales_family_monthly, kind='line')
sns.lineplot(data=sales_family_monthly.T)
plt.show()

# 期間は2017年8月15日までなので、8月の売り上げが下がっているように見えるのは妥当である。

In [ ]:
train

In [ ]:
# 2013年～2017年の日ごとの売り上げ推移をみる
train_info = train[["date", "family", "sales"]]
train_info["date"] = pd.to_datetime(train_info["date"])
sales_family_daily = pd.pivot_table(train_info, values='sales', index='family', columns='date', aggfunc='sum', fill_value=0)
sales_family_daily 

In [ ]:
plt.figure(figsize = (40,20))
plt.xticks(rotation=90)
plt.ticklabel_format(style='plain',axis='y')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.tight_layout()
sns.lineplot(data=sales_family_daily.T)
plt.show()

In [ ]:
train_daily = train_info.copy()
train_daily["year"] = train_info["date"].dt.year
train_daily

In [ ]:
train_daily[train_daily["year"] == 2013]

In [ ]:
for i in [2013, 2014, 2015, 2016, 2017]:
    train_daily_year = train_daily[train_daily["year"] == i]
    train_info = train_daily_year[["date", "family", "sales"]]
    sales_family_daily = pd.pivot_table(train_info, values='sales', index='family', columns='date', aggfunc='sum', fill_value=0)
    
    plt.figure(figsize = (20,10))
    plt.xticks(rotation=90)
    plt.ticklabel_format(style='plain',axis='y')
    sns.lineplot(data=sales_family_daily.T)
    plt.show()

In [ ]:
oil["date"] = pd.to_datetime(oil["date"])
oil

In [ ]:
plt.figure(figsize = (20,10))
plt.xticks(rotation=90)
plt.ticklabel_format(style='plain',axis='y')

sns.lineplot(data=oil.set_index("date"))
plt.show()

# 原油価格が下がることによって、輸入コストが減少するため、売り上げが増えるという仮定

In [ ]:
# 2013年～2017年の日ごとの売り上げ推移をみる
train_info = train[["date", "family", "sales"]]
train_info["date"] = pd.to_datetime(train_info["date"])
sales_family_daily = pd.pivot_table(train_info, values='sales', index='family', columns='date', aggfunc='sum', fill_value=0)
# sales_family_daily 

In [ ]:
sales_family_daily.T.index
# .T.reset_index(drop=False).set_index('date').columns

In [ ]:
!pip install japanize_matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import japanize_matplotlib
%matplotlib inline

sales_family_daily = sales_family_daily.T

fig, ax1 = plt.subplots(1,1, figsize = (60,30))
ax2 = ax1.twinx()
# 2軸グラフの本体設定
for column in sales_family_daily.columns:
    ax1.plot(sales_family_daily.index, sales_family_daily[column])

ax2.plot(oil['date'], oil['dcoilwtico'], color='000000')
#グラフタイトルを付ける
plt.title("商品ジャンルごとの日別の売り上げと石油価格の推移", fontsize=15)
# plt.figure(figsize = (40,20))
# plt.xticks(rotation=90)
# plt.ticklabel_format(style='plain',axis='y')
# plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
# 凡例の表示のため、handler1と2にはグラフオブジェクトのリスト情報が入る
# label1と2には、凡例用に各labelのリスト情報が入る
handler1, label1 = ax1.get_legend_handles_labels()
handler2, label2 = ax2.get_legend_handles_labels()
# 凡例をまとめて出力する
ax1.legend(handler1 + handler2, label1 + label2, loc=2, borderaxespad=0.)
sales_max = 100 + sales_family_daily.max().max()
oil_max = 10 + oil["dcoilwtico"].max()
ax1.set_ylim([0, sales_max])
ax2.set_ylim([0, oil_max])
# plt.show()

In [ ]:
# ①train.csv + oil.csv 
# ②train.csv + stores = .csv 
# ③train.csv + holidays_events.csv

# dateの列をstringからdatetime型に変換
oil["date"] = pd.to_datetime(oil["date"])
train["date"] = pd.to_datetime(train["date"])
holidays_events["date"] = pd.to_datetime(holidays_events["date"])

# データ作成
train = train[train['sales'] != 0]
oil = oil.set_index('date').interpolate(limit=3, limit_direction='backward')
train_oil = train.merge(oil, on="date")
train_oil_stores = train_oil.merge(store, on="store_nbr")
train_oil_stores = train_oil_stores.rename(columns={"type":"store_type"})
# train_oil_stores_holiday = train_oil_stores.merge(holidays_events, on="date")
# train_oil_stores_holiday = train_oil_stores_holiday.rename(columns={"type":"holiday_type"})
# train_oil_stores_holiday

In [ ]:
#最大表示行数の指定（ここでは50行を指定）
pd.set_option('display.max_rows', 100)
train_oil_stores

In [ ]:
# train_oil_stores_holiday["holiday_type"].value_counts()
# holiday : 通常の休日
# Event :　イベント休日？
# Additional : 追加された休日（クリスマス直前とかついで連休みたいなイメージ）
# Transfer : 移動された休日（おそらくTranseferの直線のholidayはholidayじゃなく平日に変化している）
# Bridge: 

In [ ]:
# 相関係数の考察

# 仮説立案

# 仮説検証

In [ ]:
# カテゴリ変数↓
 # family
 # city + state
 # store_type
 # type

# 保留
 # cluster
 # locale + locale_name

# 使わない
# store_nbr
# description
# transferred
# onpromotion

In [ ]:
train_oil_stores['year']  = train_oil_stores['date'].dt.year
train_oil_stores['month'] = train_oil_stores['date'].dt.month
train_oil_stores['day'] = train_oil_stores['date'].dt.day
train_oil_stores['dayofweek'] = train_oil_stores['date'].dt.dayofweek
train_oil_stores

In [ ]:
train_oil_stores = train_oil_stores.drop(["date"
                                          , "store_nbr"
#                                           , "onpromotion"
                                         ], axis=1)
train_oil_stores

In [ ]:
train_oil_stores = train_oil_stores.set_index("id")
# train_oil_stores_holiday = train_oil_stores_holiday.reset_index()
train_oil_stores

In [ ]:
# from sklearn.preprocessing import OneHotEncoder

# encoder = OneHotEncoder(sparse=False)

# arr = encoder.fit_transform(train_oil_stores_holiday[['family',
#                                                       'city', 
#                                                       'state', 
#                                                       'store_type', 
#                                                       'holiday_type', 
#                                                       'locale', 
#                                                       'locale_name']].values.reshape(-1, 1))

# df_onehot = pd.DataFrame(arr, columns=encoder.categories_)
# df_onehot

# # 列名を取得
# label = ohe.get_feature_names(['Embarked', 'Sex'])

# # データフレーム化
# df_ = pd.DataFrame(encoded, columns=label, dtype=np.int8)

# # データフレームを結合
# pd.concat([df, df_], axis=1)


In [ ]:
# col_list = ['family','city', 'state', 'store_type', 'holiday_type','locale','locale_name']
col_list = ['family','city', 'state', 'store_type', 'cluster','year', 'month']
df = pd.get_dummies(train_oil_stores, columns=col_list, sparse=True)

In [ ]:
df.columns

In [ ]:
# df = df.drop(['holiday_type_Additional',
#               'holiday_type_Bridge', 
#               'holiday_type_Event',
#               'holiday_type_Transfer'], axis=1)

In [ ]:
pd.set_option('display.max_rows', 140)
df_notnull = df[df["dcoilwtico"].notnull()]
df_notnull.isnull().sum()

In [ ]:
df_notnull

In [ ]:
X = df_notnull.iloc[:,1:]
Y = df_notnull["sales"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

# 5-fold CVモデルの学習

# 【ブロック１：　初期化】
# 5つのモデルを保存するリストの初期化
models = []

# 学習用データでの予測値を保存するデータフレームの初期化
df_train_preds = pd.DataFrame({'y_train': y_train})

# 検証用データでの予測値を保存するデータフレームの初期化
df_eval_preds = pd.DataFrame({'y_eval': [],
                              'y_eval_pred': []})

# テストデータでの予測値を保存するデータフレームの初期化
df_test_preds = pd.DataFrame({'y_test': y_test})
# インデックスが0からの連番になるように初期化

df_test_preds.reset_index(inplace=True, drop=True)

# R^2を保存するデータフレームの初期化
df_R2 = pd.DataFrame({'train': [],
                      'eval': [],
                      'test': []})

# RMSEを保存するデータフレームの初期化
df_RMSE =pd.DataFrame({'train': [],
                       'eval': [],
                       'test': []})

# ラウンド数の初期化
round_no = 0


# 【ブロック２：　モデルの学習】
# 学習データの数だけの数列（0行から最終行まで連番）
row_no_list = list(range(len(y_train)))

# KFoldクラスをインスタンス化（これを使って5分割する）
K_fold = KFold(n_splits=5, shuffle=True,  random_state=42)


# KFoldクラスで分割した回数だけ実行（ここでは5回）
for train_cv_no, eval_cv_no in K_fold.split(row_no_list, y_train):
    # ilocで取り出す行を指定
    X_train_cv = X_train.iloc[train_cv_no, :]
    y_train_cv = pd.Series(y_train).iloc[train_cv_no]
    X_eval_cv = X_train.iloc[eval_cv_no, :]
    y_eval_cv = pd.Series(y_train).iloc[eval_cv_no]
    
    # 学習用
    lgb_train = lgb.Dataset(X_train_cv, y_train_cv,
                            free_raw_data=False)
    # 検証用
    lgb_eval = lgb.Dataset(X_eval_cv, y_eval_cv, reference=lgb_train,
                           free_raw_data=False)
    
    # パラメータを設定
    params = {'task': 'train',                # 学習、トレーニング ⇔　予測predict
              'boosting_type': 'gbdt',        # 勾配ブースティング
              'objective': 'regression',      # 目的関数：回帰
              'metric': 'rmse',               # 分類モデルの性能を測る指標
              'learning_rate': 0.1 }          # 学習率（初期値0.1）
    
    # 学習
    evaluation_results = {}                              # 学習の経過を保存する箱
    model = lgb.train(params,                            # 上記で設定したパラメータ
                      lgb_train,                         # 使用するデータセット
                      num_boost_round=1000,              # 学習の回数
                      valid_names=['train', 'valid'],    # 学習経過で表示する名称
                      valid_sets=[lgb_train, lgb_eval],  # モデル検証のデータセット
                      evals_result=evaluation_results,   # 学習の経過を保存
                      early_stopping_rounds=20,          # アーリーストッピング
                      verbose_eval=0)                    # 学習の経過の非表示
    
    # 学習が終わったモデルをリストに保存
    models.append(model) 
    
    
    # 【ブロック３：　モデルで予測】
    # 学習したモデルで予測
    y_train_pred = model.predict(X_train_cv, num_iteration=model.best_iteration)
    y_eval_pred = model.predict(X_eval_cv, num_iteration=model.best_iteration)
    y_test_pred = model.predict(X_test, num_iteration=model.best_iteration)
    
    # 学習用データでの予測値をデータフレームに保存
    df_train_cv_pred = pd.DataFrame({round_no: y_train_pred},
                                     index=train_cv_no)
    df_train_preds = df_train_preds.join(df_train_cv_pred, how='left')
    
    # 検証用データでの予測値をデータフレームに保存
    df_eval_pred = pd.DataFrame({'y_eval': y_eval_cv,
                                 'y_eval_pred': y_eval_pred})
    df_eval_preds = df_eval_preds.append(df_eval_pred)    
    
    # テストデータでの予測値をデータフレームに保存
    df_test_cv_pred = pd.DataFrame({round_no: y_test_pred})
    df_test_preds = pd.concat([df_test_preds, df_test_cv_pred], axis=1)

    #  r2_score を計算
    train_score = r2_score(y_train_cv, y_train_pred)
    eval_score = r2_score(y_eval_cv, y_eval_pred)
    test_score = r2_score(y_test, y_test_pred)
    
    #  RMSE を計算
    train_RMSE_score = np.sqrt(mean_squared_error(y_train_cv, y_train_pred))
    eval_RMSE_score = np.sqrt(mean_squared_error(y_eval_cv, y_eval_pred))
    test_RMSE_score = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    # スコアを表示
    print('R^2 train: %.5f, eval: %.5f, test: %.5f' 
          % (train_score, eval_score, test_score),
         '  |  RMSE train: %.5f, eval: %.5f, test: %.5f' 
          % (train_RMSE_score, eval_RMSE_score, test_RMSE_score))
    
    # R2の保存
    df_R2_cv = pd.DataFrame({'train': [train_score],
                             'eval': [eval_score],
                             'test': [test_score]},
                             index=[round_no])
    df_R2 = df_R2.append(df_R2_cv)
    
    # RMSEの保存
    df_RMSE_cv =pd.DataFrame({'train': [train_RMSE_score],
                              'eval': [eval_RMSE_score],
                              'test': [test_RMSE_score]},
                              index=[round_no])
    df_RMSE = df_RMSE.append(df_RMSE_cv)
    
    # ラウンド数のカウンタを更新
    round_no += 1


# 保存したR2の平均値    
R2_ave = df_R2.mean().to_numpy()
# 保存したRMSEの平均値   
RMSE_ave = df_RMSE.mean().to_numpy()
# 平均値を表示
print('Average:')
print('R^2 train: %.5f, eval: %.5f, test: %.5f'
      % (R2_ave[0], R2_ave[1], R2_ave[2]),
      '  |  RMSE train: %.5f, eval: %.5f, test: %.5f'
      % (RMSE_ave[0], RMSE_ave[1], RMSE_ave[2]))

In [ ]:
# 初回
# model = create_model('lightgbm')
# パラメータ調整後
model = create_model('rf', fold=5)

In [ ]:
tuned_model = tune_model(model, optimize = 'RMSE', fold=5)

In [ ]:
# モデルの評価指標を確認
evaluate_model(tuned_model)

In [ ]:
final_model = finalize_model(tuned_model)

In [ ]:
oil

In [ ]:
# ①train.csv + oil.csv 
# ②train.csv + stores = .csv 
# ③train.csv + holidays_events.csv

# dateの列をstringからdatetime型に変換
test["date"] = pd.to_datetime(test["date"])

# データ作成
oil = oil.set_index('date').interpolate(limit=3, limit_direction='backward')
test_oil = test.merge(oil, on="date", how="inner")
test_oil_stores = test_oil.merge(store, on="store_nbr")
test_oil_stores = test_oil_stores.rename(columns={"type":"store_type"})
# test_oil_stores_holiday = test_oil_stores.merge(holidays_events, on="date", how="left")
# test_oil_stores_holiday = test_oil_stores_holiday.rename(columns={"type":"holiday_type"})
# test_oil_stores_holiday
test_oil_stores['year']  = test_oil_stores['date'].dt.year
test_oil_stores['month'] = test_oil_stores['date'].dt.month
test_oil_stores['day'] = test_oil_stores['date'].dt.day
test_oil_stores['dayofweek'] = test_oil_stores['date'].dt.dayofweek
test_oil_stores = test_oil_stores.drop(["date"
                                        , "store_nbr"
#                                         , "onpromotion"
                                       ], axis=1)
test_oil_stores = test_oil_stores.set_index("id")

# col_list = ['family','city', 'state', 'store_type', 'holiday_type','locale','locale_name']
col_list = ['family','city', 'state', 'store_type', 'cluster','year', 'month']
test_df = pd.get_dummies(test_oil_stores, columns=col_list, sparse=True)
test_df

In [ ]:
pred = predict_model(final_model, data = test_df)

In [ ]:
pred.reset_index()[["id","Label"]].rename(columns={"Label":"sales"})

In [ ]:
pred = pred.reset_index()[["id","Label"]].rename(columns={"Label":"sales"})

In [ ]:
# pred[pred['sales'] <= 0].sort_values('sales',ascending=True)

In [ ]:
pred[pred['sales'] <= 0]['sales'] = 0

pred.loc[pred['sales'] < 0, 'sales'] = 0

In [ ]:
pred

In [ ]:
sample_submission = pd.read_csv('../Dataset/store-sales-time-series-forecasting/sample_submission.csv')


In [ ]:
sample_submission

In [ ]:
test

In [ ]:
pred.to_csv('forecasting-store-sales.csv', index = False)